In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"  # !!configure environment to run tf-keras (instead of standalong Keras)
import tensorflow as tf
# import keras

# --- check version --- 
print(tf.__version__)
# print(keras.__version__)
# print("Keras path:", keras.__file__)

2.14.0


In [2]:
# --- check model (.keras as some kind of zip) --- 
import zipfile
with zipfile.ZipFile("tf_model.keras", 'r') as z:
    z.printdir()  # Lists files inside the archive

File Name                                             Modified             Size
metadata.json                                  1980-01-01 00:00:00           64
config.json                                    1980-01-01 00:00:00         6470
model.weights.h5                               2025-03-30 22:13:08        40824


#-- for debugging --#
from tensorflow.keras.models import load_model

try:
    model = load_model("tf_model.keras", compile=False)
except Exception as e:
    print("Got error:", e)

## Load Model

In [3]:
# --- load model --- 
model = tf.keras.models.load_model("tf_model.keras", compile=False)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(55, 12)]                0         
                                                                 
 batchnorm_1 (BatchNormaliz  (55, 12)                  48        
 ation)                                                          
                                                                 
 dense_1 (Dense)             (55, 64)                  832       
                                                                 
 batchnorm_2 (BatchNormaliz  (55, 64)                  256       
 ation)                                                          
                                                                 
 dropout (Dropout)           (55, 64)                  0         
                                                                 
 dense_2 (Dense)             (55, 32)                  2080  

In [4]:
# --- Plot the model architecture --- 
from tensorflow.keras.utils import plot_model
output_image_path = "layers.png"

plot_model(
    model,
    to_file=output_image_path,
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True
)

print(f"\nModel architecture plot saved as: {os.path.abspath(output_image_path)}")


Model architecture plot saved as: C:\Users\g9161\Gridware\workspace\layers.png


## TF Lite Model

In [5]:
# --- convert keras model into TF Lite --- 
TFLITE_MODEL_NAME = "tf_lite_model.tflite"
tf_lite_converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = tf_lite_converter.convert()

INFO:tensorflow:Assets written to: C:\Users\g9161\AppData\Local\Temp\tmpf6qnhl0r\assets


INFO:tensorflow:Assets written to: C:\Users\g9161\AppData\Local\Temp\tmpf6qnhl0r\assets


In [6]:
# --- write to file --- 
open(TFLITE_MODEL_NAME, "wb").write(tflite_model)

14548

In [7]:
# --- check file size --- 
tflite_file_size = os.path.getsize(TFLITE_MODEL_NAME)
def convert_bytes(size, unit=None):
    if unit == "KB":
        return print('File size: ' + str(round(size / 1024, 3)) + ' Kilobytes')
    elif unit == "MB":
        return print('File size: ' + str(round(size / (1024 * 1024), 3)) + ' Megabytes')
    else:
        return print('File size: ' + str(size) + ' bytes')
convert_bytes(tflite_file_size, "KB")

File size: 14.207 Kilobytes


## Inference TF Lite Model (Verification)

In [8]:
# --- read TF Lite Model --- 
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_NAME)
interpreter.allocate_tensors()
# --- check I/O type --- 
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input dtype:", input_details[0]['dtype'])

Input dtype: <class 'numpy.float32'>


In [9]:
# --- form input --- 
import numpy as np
test_input = [4861.92, 313.62, 0.15, -0.05, -810, 12389, 0.05, 0.1, 763.684, 12610, -0.00001093, 0.0000104]
test_input = np.tile(test_input, (55, 1)).astype('float32')  # Shape (55, 12)
# --- Inference --- 
interpreter.set_tensor(input_details[0]['index'], test_input)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print(output)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]]


## Convert to Hex Array for C++ Use

In [12]:
def convert_tflite_to_c_array(file_path, var_name="model"):
    with open(file_path, "rb") as f:
        data = f.read()
    
    with open(f"{var_name}.cc", "w") as f:
        f.write(f"const unsigned char {var_name}[] = {{\n")

        for i, byte in enumerate(data):
            if i % 12 == 0:
                f.write("\n    ")
            f.write(f"0x{byte:02x}, ")

        f.write(f"\n}};\n")
        f.write(f"const unsigned int {var_name}_len = {len(data)};\n")

# Usage
convert_tflite_to_c_array(TFLITE_MODEL_NAME, "model_tflite")